In [31]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import tensorflow as tf
from encoder_new import encoder
from decoder_new import decoder
import time

SOS = 0
EOS = 0

输入encoder—input和batch-size和模式，输出暂时未知

dataset = tf.data.Dataset.from_tensor_slices(np.random.uniform(size=(5, 2))) 
传入的数值是一个矩阵，它的形状为(5, 2)，tf.data.Dataset.from_tensor_slices就会切分它形状上的第一个维度，最后生成的dataset中一个含有5个元素，每个元素的形状是(2, )，即每个元素是矩阵的一行。
放到这里，encoder_input是一个list，里面每一个元素都代表了一个架构，因此可以看成切成了一个个架构

该函数是用来对输入进行一个预处理的函数

one_shot_iterator 返回的是一个iterator，并且数据集只能从头到尾读一次

目前只完成了这个函数一半的考察

接下来开始补mode为train时的部分其中symmetry暂时不知道干啥的，encoder-target是一个关于encoderinput对应架构精度的一个东西，decoder-target是encoderinput的拷贝

preprocess函数用来增加decoder_input, 这个decoder—input具体怎么用，未知，只知道一开始插入了一个起始标记SOS，感觉像循环神经网络的每次的input，然后输出下一个code，细节不太清楚，也不是清楚为什么可以用tf.concat([sos_id, decoder_tgt[:-1],axis=0)这样连起来，sos_id不是就只有一个0吗难道。原来decoder_tgt[:-1]现在也就是一个一维的张量而已。这就解释了为什么可以concat。但是如何处理这个decoder_input还得从后序代码中理解

generate_symmetry是根据一个结构生成镜像结构是函数，详细说，操作就是随机挑出某一个(I1,OP1,I2,OP2)转换成(I2,OP2,I1,OP1)

In [32]:
def input_fn(encoder_input, encoder_target, decoder_target, mode, batch_size, num_epochs=1, symmetry=False):
    shape = np.array(encoder_input).shape
    if mode == 'train':
        tf.logging.info('Data size: {}, {}'.format(shape, np.array(encoder_target).shape))
        N = shape[0]
        source_length = shape[1]
        encoder_input = tf.convert_to_tensor(encoder_input, dtype=tf.int32)
        encoder_input = tf.data.Dataset.from_tensor_slices(encoder_input)
        encoder_target = tf.convert_to_tensor(encoder_target, dtype=tf.float32)
        encoder_target = tf.data.Dataset.from_tensor_slices(encoder_target)
        decoder_target = tf.convert_to_tensor(decoder_target, dtype=tf.int32)
        decoder_target = tf.data.Dataset.from_tensor_slices(decoder_target)
        dataset = tf.data.Dataset.zip((encoder_input, encoder_target, decoder_target))
        dataset = dataset.shuffle(buffer_size=N)
        
        def preprocess(encoder_src, encoder_tgt, decoder_tgt): #src:sequence tgt:performance
            sos_id = tf.constant([SOS])
            decoder_src = tf.concat([sos_id, decoder_tgt[:-1]], axis=0)
            return (encoder_src, encoder_tgt, decoder_src, decoder_tgt)
        def generate_symmetry(encoder_src, encoder_tgt, decoder_src, decoder_tgt):
            a = tf.random_uniform([], 0, 5, dtype=tf.int32)
            b = tf.random_uniform([], 0, 5, dtype=tf.int32)
            cell_seq_length = source_length // 2
            assert source_length in [40, 60]
            if source_length == 40:
                encoder_src = tf.concat([encoder_src[:4 * a], encoder_src[4*a+2 : 4*a+4], encoder_src[4*a:4*a+2], encoder_src[4*(a+1):cell_seq_length+4*b], encoder_src[cell_seq_length+4*b+2:cell_seq_length+4*b+4], encoder_src[cell_seq_length+4*b:cell_seq_length+4*b+2], encoder_src[cell_seq_length+4*(b+1):]],axis=0)
            else:#source_length=60
                encoder_src = tf.concat([encoder_src[:6 * a], encoder_src[6*a+3 : 6*a+6], encoder_src[6*a:6*a+3], encoder_src[6*(a+1):cell_seq_length+6*b], encoder_src[cell_seq_length+6*b+3:cell_seq_length+6*b+6], encoder_src[cell_seq_length+6*b:cell_seq_length+6*b+3], encoder_src[cell_seq_length+6*(b+1):]],axis=0)
                
            decoder_tgt = encoder_src
                
            return encoder_src, encoder_tgt, decoder_src, decoder_tgt
        
        dataset = dataset.map(preprocess)
        if symmetry:
            dataset = dataset.map(generate_symmetry)
        dataset = dataset.repeat(num_epochs)
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(10)
        iterator = dataset.make_one_shot_iterator()
        encoder_input, encoder_target, decoder_input, decoder_target = iterator.get_next()
        assert encoder_input.shape.ndims == 2
        assert encoder_target.shape.ndims == 1
        while encoder_target.shape.ndims < 2:
            encoder_target = tf.expand_dims(encoder_target, axis=-1)
        assert decoder_input.shape.ndims == 2
        assert decoder_target.shape.ndims == 2
        return encoder_input, encoder_target, decoder_input, decoder_target
        
        
    else:
        tf.logging.info("Data size : {}".format(shape))
        encoder_input = tf.convert_to_tensor(encoder_input, dtype=tf.int32)
        encoder_input = tf.data.Dataset.from_tensor_slices(encoder_input)
        def preprocess(encoder_src):
            return encoder_src, tf.constant([SOS], dtype=tf.int32)
        dataset = encoder_input.map(preprocess)
        dataset = dataset.repeat(num_epochs)
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(10)
        iterator = dataset.make_one_shot_iterator()
        encoder_input, decoder_input = iterator.get_next()
        assert encoder_input.shape.ndims == 2
        return encoder_input, decoder_input
        
        
        

输入encoder input target加上decoder input target以及controller的参数，输出训练所需要的一些操作

In [ ]:
def get_train_ops(encoder_train_input, encoder_train_target, decoder_train_input, decoder_train_target, params, reuse=tf.AUTO_REUSE):
    
    with tf.variable_scope('EPD', reuse=reuse):
        my_encoder = encoder.Model(
            encoder_train_input,
            encoder_train_target,
            params,
            tf.estimator.ModeKeys.TRAIN,
            'Encoder',
            reuse
        )
        encoder_outputs = my_encoder.encoder_outputs
        encoder_state = my_encoder.arch_emb #shape = [batch_size, encoder_hidden_size]
        encoder_state.set_shape([None, params['decoder_hidden_size']]) 
        encoder_state = tf.contrib.rnn.LSTMStateTuple(encoder_state, encoder_state)
        encoder_state = (encoder_state,) * params['decoder_num_layers']
        my_decoder = decoder.Model(encoder_outputs, encoder_state, decoder_train_input, decoder_train_target,params,tf.estimator.ModeKeys.TRAIN,'Decoder',reuse)
        encoder_loss = my_encoder.loss
        decoder_loss = my_decoder.loss
        mse = encoder_loss
        cross_entropy = decoder_loss
        
        total_loss = params['trade_off'] * encoder_loss + (1 - params['trade_off']) * decoder_loss + params['weight_decay'] * tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])
        
        tf.summary.scalar('training_loss', total_loss)
        
        global_step = tf.train.get_or_create_global_step()
        learning_rate = tf.constant(params['lr'])
        
        if params['optimizer'] == 'sgd':
            learning_rate = tf.cond(
                global_step < params['start_decay_step'],
                lambda: learning_rate,
                lambda: tf.train.exponential_decay(
                    learning_rate,
                    (global_step - params['start_decay_step']),
                    params['decay_steps'],
                    params['decay_factor'],
                    staircase=True),
                name='calc_learning_rate')
            
            opt = tf.train.GradientDescentOptimizer(learning_rate)
        elif params['optimizer'] == 'adam':
            assert float(params['lr']) <= 0.001, "! High Adam learning rate %g" % params['lr']
            opt = tf.train.AdamOptimizer(learning_rate)
        elif params['optimizer'] == 'adadelta':
            opt = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
        tf.summary.scalar("learning_rate", learning_rate)
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        
        with tf.control_dependencies(update_ops):
            gradients, variables = zip(*opt.compute_gradients(total_loss))
            grad_norm = tf.global_norm(gradients)
            clipped_gradients,_ = tf.clip_by_global_norm(gradients, params['max_gradient_norm'])
            
            train_op = opt.apply_gradients(zip(clipped_gradients,variables), global_step=global_step)
            
        return mse, cross_entropy, total_loss, learning_rate, train_op, global_step, grad_norm
    
                

这个函数输入处理完成的encoder-input和decoder-input以及控制组件的参数，返回的是predict-value，sample-id，new-sample-id

训练模式，即 mode == tf.estimator.ModeKeys.TRAIN，必须提供的是 loss 和 train_op。
验证模式，即 mode == tf.estimator.ModeKeys.EVAL，必须提供的是 loss。
预测模式，即 mode == tf.estimator.ModeKeys.PREDICT，必须提供的是 predicitions。


未完成：
my_encoder.infer()

my_decoder.decode()已实现

In [ ]:
def get_predict_ops(encoder_predict_input, decoder_predict_input, params, reuse=tf.AUTO_REUSE):
    encoder_predict_target = None
    decoder_predict_target = None
    with tf.variable_scope('EPD', reuse=reuse):
        my_encoder = encoder.Model(encoder_predict_input, encoder_predict_target, params, tf.estimator.ModeKeys.PREDICT,'Encoder',reuse)
        encoder_outputs = my_encoder.encoder_outputs
        encoder_state = my_encoder.arch_emb
        encoder_state.set_shape([None, params['decoder_hidden_size']])
        encoder_state = tf.contrib.rnn.LSTMStateTuple(encoder_state, encoder_state)
        encoder_state = (encoder_state, ) * params['decoder_num_layers']
        my_decoder = decoder.Model(encoder_outputs, encoder_state, decoder_predict_input,decoder_predict_target, params, tf.estimator.ModeKeys.PREDICT, "Decoder",reuse)
        
        arch_emb, predict_value, new_arch_emb, new_arch_outputs = my_encoder.infer()
        
        sample_id = my_decoder.decode()
        
        encoder_state = new_arch_emb
        encoder_state.set_shape([None, params['decoder_hidden_size']])
        encoder_state = tf.contrib.rnn.LSTMStateTuple(encoder_state,encoder_state)
        encoder_state = (encoder_state, )* params['decoder_num_layers']
        tf.get_variable_scope().reuse_variables()
        my_decoder = decoder.Model(new_arch_outputs, encoder_state,decoder_predict_input, decoder_predict_target,params, tf.estimator.ModeKeys.PREDICT,'Decoder')
        new_sample_id = my_decoder.decode()
        
        return predict_value, sample_id, new_sample_id
        
        

输入训练参数，encoder_input, encoder_target,decoder_target来训练Encoder-Predictor-Decoder

get_train_ops用来返回训练时所需要的一些操作，已经实现

tf.summary_merge_all()是可视化相关的一个操作，以便tensorboard显示

tf.summary.FileWriter 指定一个文件用来保存图

train_writer.add_summary(train_summary,step)#调用train_writer的add_summary方法将训练过程以及训练步数保存 

In [ ]:
def train(params, encoder_input, encoder_target, decoder_target):
    with tf.Graph().as_default():
        tf.logging.info('Training Encoder-Predictor-Decoder')
        tf.logging.info('Preparing data')
        shape = np.array(encoder_input).shape
        N = shape[0]
        encoder_train_input, encoder_train_target, decoder_train_input, decoder_train_target = input_fn(
            encoder_input,
            encoder_target,
            decoder_target,
            'train',
            params['batch_size'],
            None,
            params['symmetry'],
        )
        
        tf.logging.info('Building model')
        train_mse, train_cross_entropy, train_loss, learning_rate, train_op, global_step, grad_norm = get_train_ops(
            encoder_train_input,
            encoder_train_target,
            decoder_train_input,
            decoder_train_target,
            params
        )
        saver = tf.train.Saver(max_to_keep=10)
        checkpoint_saver_hook = tf.train.CheckpointSaverHook(
            params['model_dir'], 
            save_steps=params['batches_per_epoch']*params['save_frequency'],
            saver=saver
        )
        hooks = [checkpoint_saver_hook]
        merged_summary = tf.summary.merge_all()
        tf.logging.info('Starting Session')
        config = tf.ConfigProto(allow_soft_placement=True)
        with tf.train.SingularMonitoredSession(config=config, hooks=hooks, checkpoint_dir=params['model_dir']) as sess:
            writer = tf.summary.FileWriter(params['model_dir'], sess.graph)
            start_time = time.time()
            for step in range(int(params['train_epochs'] * params['batches_per_epoch'] )):
                run_ops = [
                    train_mse,
                    train_cross_entropy,
                    train_loss,
                    learning_rate,
                    train_op,
                    global_step,
                    grad_norm,
                    merged_summary,
                    
                ]
                
                train_mse_v, train_cross_entropy_v, train_loss_v, learning_rate_v, _, global_step_v, gn_v, summary = sess.run(run_ops)
                
                writer.add_summary(summary, global_step_v)
                
                epoch = (global_step_v+1) // params['batches_per_epoch']
                
                curr_time = time.time()
                
                if (global_step_v+1)%100 == 0:
                    log_string = "epoch={:<6d} ".format(int(epoch))
                    log_string += "step={:<6d} ".format(int(global_step_v+1))
                    log_string += "se={:<6f} ".format(train_mse_v)
                    log_string += "cross_entropy={:<6f} ".format(train_cross_entropy_v)
                    log_string += "loss={:<6f} ".format(train_loss_v)
                    log_string += "learning_rate={:<8.4f} ".format(learning_rate_v)
                    log_string += "|gn|={:<8.4f} ".format(gn_v)
                    log_string += "mins={:<10.2f} ".format((curr_time - start_time) / 60)
                    tf.logging.info(log_string)

给定controller的一些参数，给定一个列表，列表里每个元素代表一个架构的code，输出的应该是各个架构对应的预测错误率

有点不太明白这个函数的输出代表什么意思，因此暂时搁置该函数

该函数的输出是新的架构，对每个输入的架构，进行更新，更新完成之后，得到的新的一系列架构new_sample_id_list

In [30]:
def predict(params, encoder_input):
    with tf.Graph().as_default():
        tf.logging.info("Generating new architectures using gradient desent with step size {}".format(params['predict_lambda']))
        tf.logging.info('Preparing data')
        N = len(encoder_input)
        encoder_input, decoder_input = input_fn(encoder_input, None, None, 'test', params['batch_size'],1,False)
        predict_value, sample_id, new_sample_id = get_predict_ops(encoder_input, decoder_input, params)
        tf.logging.info("Starting Session")
        config = tf.ConfigProto(allow_soft_placement=True)
        new_sample_id_list=[]
        with tf.train.SingularMonitoredSession(config=config, checkpoint_dir=params['model_dir']) as sess:
            for _ in range(N // params['batch_size']):
                new_sample_id_v = sess.run(new_sample_id)
                new_sample_id_list.extend(new_sample_id_v.tolist())
        return new_sample_id_list
        